# Data Manipulation and Cleaning

In this notebook, we cleaned and manipulated our data, whichw as saved to a csv filed labeled: 'testing_data.csv'

In [2]:
#imported necessary packages

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import seaborn as sns
import statsmodels
from statsmodels.formula.api import ols

In [3]:
data = pd.read_csv('data/kc_house_data.csv')

In [4]:
data.yr_renovated.fillna(0, inplace=True)
data.waterfront.fillna('NO', inplace=True)
data.view.fillna('NONE', inplace = True)

In [5]:
data.drop(columns=['id', 'date'], inplace=True)

In [6]:
#cities and corresponding zipcodes data
cities = {'Algona': [98001], 'Auburn': [98001, 98002, 98003, 98023, 98063, 98071, 98092, 98093], 'Federal Way': [98001, 98003, 98023, 98063, 98093],
         'Beaux Arts Village': [98004], 'Bellevue':[98004, 98005, 98006, 98007, 98008, 98009, 98015], 
          'Clyde Hill': [98004], 'Hunts Point': [98004], 'Yarrow Point': [98004], 'Black Diamond': [98010],
         'Bothell': [98011, 98041, 98028], 'Burton': [98013], 'Vashion': [98013], 'Carnation': [98014],
         'Duvall':[98019], 'Enumclaw': [98022], 'Fall City': [98024], 'Hobart': [98025], 
         'Issaquah': [98027], 'Kent': [98030, 98031, 98032, 98035, 98042, 98064], 'Kirkland':[98033, 98034, 98083],
         'Maple Valley': [98038], 'Medina': [98039], 'Mercer Island': [98040], 'Kenmore': [98028],
         'Covington':[98042], 'North Bend': [98045], 'Pacific':[98047], 'Preston':[98050],
         'Ravensdale':[98051], 'Redmond': [98052, 98053, 98073, 98074], 'Redondo': [98054], 'Renton': [98055, 98056, 98057, 98058, 98059],
         'Newcastle': [98056, 98059,], 'Seahurst':[98062], 'Snoqualmie': [98065, 98068], 'Snoqualmie Pass': [98068],
         'Vashon': [98070], 'Woodinville': [98072], 'Sammamish':[98075, 98075], 'Issaquah': [98075, 98027, 98029],
         'Seattle': [98101, 98102, 98103, 98104, 98105, 98106, 98107, 98108, 98109, 98111, 98112, 98114, 98115, 98116, 98117, 98118, 98119, 98121, 98122, 98124, 98125, 
                     98126, 98131, 98132, 98133, 98134, 98136, 98138, 98144, 98145, 98146, 98148, 98154, 98155, 98158, 98160, 98161, 98164, 98166, 98168, 98171, 98174, 98177, 98178, 98188, 98198, 98199 ], 
          'Tukwila': [98108, 98138, 98168, 98178, 98188],
         'Shoreline': [98133, 98155, 98177 ], 'Burien': [98146, 98148, 98166, 98168 ], 'Des Moines' : [98148, 98198], 'Normandy Park': [98148, 98166, 98198], 'Seatac': [98148, 98158, 98168, 98188, 98198], 
          'Lake Forest Park': [98155, 98155, 98155], 'Baring': [98224], 'Skykomish': [98288]}

In [7]:
def city_finder(zipcode):
    
    """
    This function assigns zipcodes to their corresponding city(ies)
    
    """
    
    
    lst = []
    for key, value in cities.items():
        if zipcode in value:
            lst.append(key)
    
    if lst != []:
        return lst
    else:
        return None

In [8]:
data['cities'] = data.zipcode.apply(lambda x: city_finder(x))
data.cities.isna().sum()

198

In [9]:
data.dropna(inplace=True)

In [10]:
def binary_cities(lst, city):   
    """
    
    This function takes two arguments: the list of genres from the movie.genre column in each row
    and a specify genre. Ex: binary_cities(['Seattle', 'Algona'], 'Seattle')
    
    Returns 1 if city in lst
    Returns 0 if city not in lst
    
    
    """
    
    if city in lst:
        return 1
    else:
        return 0
    
#looping through city columns to apply binary_cities function to add new columns
for i in cities.keys():
    data[i] = data.cities.apply(lambda x: binary_cities(x, i))

In [11]:
#deleted city columns that had sum of 0
deleted_list = []
for i in cities.keys():
    if data[i].sum() == 0:
        deleted_list.append(i)
        del data[i]
    else:
        continue

for i in deleted_list:
    cities.pop(i)

In [12]:
data.waterfront.replace('NO', 0, inplace=True)
data.waterfront.replace('YES', 1, inplace=True)

data.sqft_basement.replace('?', '0', inplace=True)
data.sqft_basement = data.sqft_basement.astype('float')
data['has_basement'] = data.sqft_basement.apply(lambda x: 0 if x == 0.0 else 1)

data['recently_renovated'] = data.yr_renovated.apply(lambda x: 0 if x < 2000 else 1)

data.grade = data.grade.astype('category')
data.grade = data.grade.cat.reorder_categories(['3 Poor', '4 Low', '5 Fair', '6 Low Average', '7 Average', '8 Good',
                                                '9 Better', '10 Very Good', '11 Excellent', '12 Luxury', '13 Mansion'])

data.condition = data.condition.astype('category')
data.condition = data.condition.cat.reorder_categories(['Fair', 'Poor', 'Average',
                                                        'Good', 'Very Good'])

data.view = data.view.astype('category')
data.view = data.view.cat.reorder_categories(['NONE', 'FAIR', 'AVERAGE', 'GOOD', 'EXCELLENT'])

In [13]:
sum_list = []
for i in cities.keys():
    print('{} : {}'.format(i, data[i].sum()))
    sum_list.append(data[i].sum())
    
    
city_dfs = {}

for i in cities.keys():
    
    df = data[data[i] == 1]
    city_dfs[i] = df

Algona : 361
Auburn : 1690
Federal Way : 1140
Beaux Arts Village : 317
Bellevue : 1407
Clyde Hill : 317
Hunts Point : 317
Yarrow Point : 317
Black Diamond : 100
Bothell : 478
Carnation : 124
Duvall : 190
Enumclaw : 233
Fall City : 80
Issaquah : 1092
Kent : 1201
Kirkland : 977
Maple Valley : 589
Medina : 50
Mercer Island : 282
Kenmore : 283
Covington : 547
North Bend : 220
Redmond : 1418
Renton : 1597
Newcastle : 874
Snoqualmie : 308
Vashon : 117
Woodinville : 273
Sammamish : 359
Seattle : 8973
Tukwila : 853
Shoreline : 1194
Burien : 868
Des Moines : 337
Normandy Park : 591
Seatac : 742
Lake Forest Park : 446


In [14]:
#seems like hunts point yarrow point clyde hill and beaux arts village might all be the same place

data[(data['Beaux Arts Village'] == 1) & (data['Clyde Hill'] == 1) & (data['Hunts Point'] ==1) & (data['Yarrow Point']) ==1]

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,Seattle,Tukwila,Shoreline,Burien,Des Moines,Normandy Park,Seatac,Lake Forest Park,has_basement,recently_renovated
66,975000.0,4,2.50,2720,11049,2.0,0,NONE,Average,10 Very Good,...,0,0,0,0,0,0,0,0,0,0
69,1330000.0,5,2.25,3200,20158,1.0,0,NONE,Average,8 Good,...,0,0,0,0,0,0,0,0,1,0
103,1090000.0,3,2.50,2920,8113,2.0,0,NONE,Average,8 Good,...,0,0,0,0,0,0,0,0,0,1
125,1450000.0,4,2.75,2750,17789,1.5,0,NONE,Average,8 Good,...,0,0,0,0,0,0,0,0,1,0
269,2900000.0,4,3.25,5050,20100,1.5,0,AVERAGE,Average,11 Excellent,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21457,1750000.0,4,2.75,3560,8975,2.0,0,NONE,Average,10 Very Good,...,0,0,0,0,0,0,0,0,0,0
21514,3000000.0,4,3.75,5090,14823,1.0,0,NONE,Average,11 Excellent,...,0,0,0,0,0,0,0,0,1,0
21517,999999.0,3,2.50,2100,4097,2.0,0,NONE,Average,9 Better,...,0,0,0,0,0,0,0,0,0,0
21552,1700000.0,4,3.50,3830,8963,2.0,0,NONE,Average,10 Very Good,...,0,0,0,0,0,0,0,0,1,0


In [15]:
#Yarrow Point will represent these three cities
city_dfs.pop('Beaux Arts Village')
city_dfs.pop('Clyde Hill')
city_dfs.pop('Hunts Point')

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,Seattle,Tukwila,Shoreline,Burien,Des Moines,Normandy Park,Seatac,Lake Forest Park,has_basement,recently_renovated
66,975000.0,4,2.50,2720,11049,2.0,0,NONE,Average,10 Very Good,...,0,0,0,0,0,0,0,0,0,0
69,1330000.0,5,2.25,3200,20158,1.0,0,NONE,Average,8 Good,...,0,0,0,0,0,0,0,0,1,0
103,1090000.0,3,2.50,2920,8113,2.0,0,NONE,Average,8 Good,...,0,0,0,0,0,0,0,0,0,1
125,1450000.0,4,2.75,2750,17789,1.5,0,NONE,Average,8 Good,...,0,0,0,0,0,0,0,0,1,0
269,2900000.0,4,3.25,5050,20100,1.5,0,AVERAGE,Average,11 Excellent,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21457,1750000.0,4,2.75,3560,8975,2.0,0,NONE,Average,10 Very Good,...,0,0,0,0,0,0,0,0,0,0
21514,3000000.0,4,3.75,5090,14823,1.0,0,NONE,Average,11 Excellent,...,0,0,0,0,0,0,0,0,1,0
21517,999999.0,3,2.50,2100,4097,2.0,0,NONE,Average,9 Better,...,0,0,0,0,0,0,0,0,0,0
21552,1700000.0,4,3.50,3830,8963,2.0,0,NONE,Average,10 Very Good,...,0,0,0,0,0,0,0,0,1,0


In [16]:
len(city_dfs.keys())

35

##### testing_data is used in the coordinates file to do model building and testing

In [17]:
data.to_csv('data/testing_data.csv', header=True)